In [111]:
# Importar librearias

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as scistat
import pprint

# Importar tablas

d21 = pd.read_csv("/Users/oliver/Documents/Data Analyst Project/Projecto de crimenes CABA 2.0/Csv/delitos_2021.csv", sep = ';')
d20 = pd.read_csv("/Users/oliver/Documents/Data Analyst Project/Projecto de crimenes CABA 2.0/Csv/delitos_2020.csv", sep = ',')
d19 = pd.read_csv("/Users/oliver/Documents/Data Analyst Project/Projecto de crimenes CABA 2.0/Csv/delitos_2019.csv", sep = ',',dtype={'franja_horaria': object})
d18 = pd.read_csv("/Users/oliver/Documents/Data Analyst Project/Projecto de crimenes CABA 2.0/Csv/delitos_2018.csv", sep = ',')
d17 = pd.read_csv("/Users/oliver/Documents/Data Analyst Project/Projecto de crimenes CABA 2.0/Csv/delitos_2017.csv", sep = ',')



pob = pd.read_csv("/Users/oliver/Documents/Data Analyst Project/Projecto de crimenes CABA 2.0/Csv/caba_pob_barrios_2010.csv")
    

# Crear lista con las columnas a eliminar

list = ["anio", "mes", "dia"]

d21 = d21.drop(list, axis=1)

d20 = d20.drop(list, axis=1)

# Agregar columna de "uso_armas" a la tabla 19, 18 y 17 con valor Nulo

d19['uso_armas'] = np.nan

d18['uso_armas'] = np.nan

d17['uso_armas'] = np.nan

# corregir los nombres de las columnas para poder unir las tablas
# Crear dos listas con nombres de columnas, una con los nombres actuales y otra con los nuevos nombres que van a ser identicos en las demas tablas
# Crear una funcion definida de usuario para incertar las las tablas y las listas y lograr los cambios.

olist1 = d21.columns.tolist()
olist2 = d20.columns.tolist()
olist3 = d19.columns.tolist()
olist4 = pob.columns.tolist()


nlist1 = ["id", "fecha", "horario", "tipo", "subtipo", "uso_armas", "barrio", "comuna", "latitud", "longitud", "victimas"]
nlist2 = ["id", "fecha", "horario", "tipo", "subtipo", "victimas", "comuna", "barrio", "latitud", "longitud", "uso_armas"]
nlist3 = ["barrio","poblacion"]

def rename_column_name(df, old_column_list, new_column_list): 
    df = df.rename(columns=dict(zip(old_column_list, new_column_list))) 
    return df                                                                                                                                                                                               

d21 = rename_column_name(d21,olist1,nlist1)
d20 = rename_column_name(d20,olist2,nlist1)
d19 = rename_column_name(d19,olist3,nlist2)
d18 = rename_column_name(d18,olist3,nlist2)
d17 = rename_column_name(d17,olist3,nlist2)
pob = rename_column_name(pob,olist4,nlist3)

d21["fecha"] = pd.to_datetime(d21['fecha'], format='%d/%m/%Y')

# Unir verticalmente las tablas para proceder a limpiar los datos

td = pd.concat([d21, d20, d19, d18, d17])

# Reordenar el indice con valores unicos

td = td.reset_index(drop=True)

# No existen casos sin victimas por ende se le asgina 1 a los casos NaN

td.victimas.fillna(1, inplace=True)


# La siguiente funcion de usuario nos permite ver una descripcion detallada y dinámica de nuestra tabla

def getfirstlook(df, nrows=5, uniqueids=None):
    out = {}
    out['head'] = df.head(nrows)
    out['dtypes'] = df.dtypes 
    out['null'] = df.isnull().sum(axis=0)* 100 / len(df) #Porcentaje de valores nulos de la columna.
    out['nrows'] = df.shape[0]
    out['ncols'] = df.shape[1]
    out['index'] = df.index
    if (uniqueids is not None):
        out['uniqueids'] = df[uniqueids].nunique()
    return out

def displaydict(dicttodisplay):
    print(*(': '.join(map(str, x)) \
       for x in dicttodisplay.items()), sep='\n\n')

# En la función, introduciendo una columna en uniqueids nos va a informar la cantidad de valores unicos de x columna.

dfinfo = getfirstlook(td, 3, "id")
displaydict(dfinfo)

# A continuacuión podemos comprobar si cierta columna tiene valores unicos

dfinfo['nrows'] == dfinfo['uniqueids']

# Nota importante: Columna "id" no contiene valores unicos, esta no unicidad se arrastra desde la fuente.


head:                 id                fecha horario       tipo         subtipo  \
0  HC-2021-1008202  2021-01-06 00:00:00      18  Homicidio  Siniestro Vial   
1  HC-2021-1009094  2021-01-07 00:00:00      10  Homicidio  Siniestro Vial   
2  HC-2021-1013236  2021-01-10 00:00:00       7  Homicidio  Siniestro Vial   

  uso_armas            barrio  comuna       latitud      longitud  victimas  
0       NaN      Constitución     1.0     -34,62761    -58,379146       1.0  
1       NaN         Mataderos     9.0  -34,64972969  -58,51058958       1.0  
2       NaN  Parque Chacabuco     7.0  -34,62917196  -58,43058079       1.0  

dtypes: id            object
fecha         object
horario       object
tipo          object
subtipo       object
uso_armas     object
barrio        object
comuna       float64
latitud       object
longitud      object
victimas     float64
dtype: object

null: id            0.000000
fecha         0.000000
horario       0.000000
tipo          0.000000
subtipo      85.

In [38]:
# Corregir la columna "comuna" y "victimas"

convert = lambda x: '%.0f' % x if isinstance(x,float) else x
td['comuna'] = td['comuna'].apply(convert)

convert = lambda x: '%.f' % x if isinstance(x,float) else x
td['victimas'] = td['victimas'].apply(convert)

td = td.replace('nan', np.nan)

# Analizar si hay columnas con mas de un tipo de dato

# Este codigo permite analizar todos los tipos de datos de todas las columnas de la tabla y la cantidad-

td.applymap(type).apply(pd.value_counts).fillna(0)



,id,fecha,horario,tipo,subtipo,uso_armas,barrio,comuna,latitud,longitud,victimas
<class 'int'>,366781.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<class 'str'>,155292.0,433506.0,522073.0,522073.0,74952.0,15753.0,512652.0,513169.0,155292.0,155292.0,522073.0
<class 'pandas._libs.tslibs.timestamps.Timestamp'>,0.0,88567.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<class 'float'>,0.0,0.0,0.0,0.0,447121.0,506320.0,9421.0,8904.0,366781.0,366781.0,0.0


In [112]:
# Fijarse que tipo de dato es incoherente y que valor ocupa en la tabla 

td[td.latitud.apply(type) == str]

# Corregir valores nulos en toda la tabla

lisst = ["SD", "sd", "S/D", "nan"]

td = td.replace(lisst, np.nan)

# Primero pasar las columnas a str y luego cambiar las comas por puntos en las latitudes y longitudes

td['latitud'] = td['latitud'].astype(str)
td['longitud'] = td['longitud'].astype(str)


td["latitud"] = td['latitud'].str.replace(',','.')
td["longitud"] = td['longitud'].str.replace(',','.')
td["comuna"] = td['comuna'].replace("0",np.nan)

# Pasar las latitudes y longitudes al formato correcto

td['latitud'] = td.apply(lambda x: float(x['latitud']),axis=1)
td['longitud'] = td.apply(lambda x: float(x['longitud']),axis=1)

# fechas

td["fecha"] = pd.to_datetime(td['fecha'])

# Victimas tipo de dato

td['victimas'] = td['victimas'].astype(int)

# Normalizar

td["barrio"] = td["barrio"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.title()

# Corregir la tabla de poblacion y unirla con la tabla general

pob["barrio"] = pob["barrio"].str.title()

pob['barrio'].replace(['Boca'],
                   ['La Boca'], inplace=True)

td['barrio'].replace(['Nunez', "Boca"],
                   ['Nuñez', "La Boca"], inplace=True)

td = pd.merge(td, pob, how='outer', on='barrio')

# Datos categoricos

categ = ["horario", "tipo", "subtipo", "uso_armas", "comuna", "barrio"]

td[categ] = td[categ].astype("category")

td['Tasa de criminalidad cada 100k'] = (td['victimas']/td['poblacion']) * 100000

# Fin de la limpieza de datos
# Uso de memoria 30.9+ MB

td.to_csv("/Users/oliver/Documents/Data Analyst Project/Projecto de crimenes CABA 2.0/Csv/est_criminal_limpio.csv", index = True)